### 

In [1]:
#Required libraries for the project
%pip install python-dotenv
%pip install arxiv requests PyPDF2 cortex
%pip install streamlit
%pip install requests
#Required libraries for text and image extraction
%pip install "unstructured[all]" 
%pip install pdfminer.six==20231228
%pip install unstructured_inference
%pip install pytesseract
%pip install pymupdf
%pip install pi-heif
#Required libraries for snowflake
%pip install snowflake-connector-python
%pip install snowflake-snowpark-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install unstructured_pytesseract

  Using cached unstructured.pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached unstructured.pytesseract-0.3.13-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import streamlit as st
from snowflake.snowpark import Session
import snowflake.connector as sf
from dotenv import load_dotenv
import os
import shutil
import requests
import arxiv
from PyPDF2 import PdfReader
from snowflake.snowpark.types import StructType, StructField, StringType, IntegerType

In [17]:
# Load environment variables from .env file
load_dotenv()



# snowflake_account = os.getenv("SNOWFLAKE_ACCOUNT")
# snowflake_user = os.getenv("SNOWFLAKE_USER")
# snowflake_password = os.getenv("SNOWFLAKE_PASSWORD")
# snowflake_role = os.getenv("SNOWFLAKE_ROLE")
# snowflake_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
# snowflake_database = os.getenv("SNOWFLAKE_DATABASE")
# snowflake_schema = os.getenv("SNOWFLAKE_SCHEMA")

# Access secrets securely
# snowflake_account = st.secrets["SNOWFLAKE_ACCOUNT"]
# snowflake_user = st.secrets["SNOWFLAKE_USER"]
# snowflake_password = st.secrets["SNOWFLAKE_PASSWORD"]
# snowflake_role = st.secrets["SNOWFLAKE_ROLE"]
# snowflake_warehouse = st.secrets["SNOWFLAKE_WAREHOUSE"]
# snowflake_database = st.secrets["SNOWFLAKE_DATABASE"]
# snowflake_schema = st.secrets["SNOWFLAKE_SCHEMA"]

In [9]:
# Function to fetch papers from arXiv API
def fetch_arxiv_papers(keyword, max_results=5):
    search = arxiv.Search(
        query=keyword,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    papers = []
    for result in search.results():
        papers.append({
            "title": result.title,
            "authors": [author.name for author in result.authors],
            "summary": result.summary,
            "pdf_url": result.pdf_url
        })
    return papers

In [10]:
# Function to clear the downloads folder
def clear_downloads_folder(output_dir):
    if os.path.exists(output_dir):
        # Remove all files and subdirectories in the folder
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)  # Recreate the folder

# Function to download PDF and extract text
def download_and_extract_pdf(pdf_url, output_dir="downloads"):
    os.makedirs(output_dir, exist_ok=True)
    pdf_name = pdf_url.split("/")[-1] + ".pdf"
    pdf_path = os.path.join(output_dir, pdf_name)

    # Save the PDF locally
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(pdf_path, "wb") as f:
            f.write(response.content)

        # Extract text from the PDF
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

        # Clean up text
        text = " ".join(text.split())  # Remove excessive whitespace
        return pdf_path, text
    else:
        print(f"Failed to download {pdf_url} with status code {response.status_code}")
        return None, None

In [11]:

keyword = "electron"  # Example keyword
max_results = 3  # Number of papers to fetch

# Step 1: Fetch papers from arXiv
papers = fetch_arxiv_papers(keyword, max_results)

# Step 2: Clear downloads folder and then download and process PDFs
output_dir = "downloads"
clear_downloads_folder(output_dir)
pdf_paths = []
for paper in papers:
    print(f"Processing: {paper['title']}")
    pdf_path, text = download_and_extract_pdf(paper["pdf_url"], output_dir)
    if pdf_path:
        pdf_paths.append(pdf_path)
        print(f"Saved PDF: {pdf_path}")
        print(f"Extracted text (truncated): {text[:500]}...")  # Display first 500 characters of text
    else:
        print(f"Failed to process {paper['title']}")


/var/folders/kf/zy5yhv3j6_xd7bjfyt4t2vth0000gn/T/ipykernel_50995/4284433572.py:9: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Processing: Impact of Electron-Electron Cusp on Configuration Interaction Energies
Saved PDF: downloads/0102536v1.pdf
Extracted text (truncated): arXiv:cond-mat/0102536v1 [cond-mat.str-el] 28 Feb 2001Impact of Electron-Electron Cusp on Conﬁguration Interact ion Energies David Prendergast,a)M. Nolan,b)Claudia Filippi,a)Stephen Fahy,a)and J.C. Greerb) a)Department of Physics andb)NMRC, University College, Cork, Ireland (November 26, 2024) The eﬀect of the electron-electron cusp on the convergence o f conﬁguration interaction (CI) wave functions is examined. By analogy with the pseudopotential approach for electron-ion interactions, an eﬀect...
Processing: Electron thermal conductivity owing to collisions between degenerate electrons
Saved PDF: downloads/0608371v1.pdf
Extracted text (truncated): arXiv:astro-ph/0608371v1 17 Aug 2006Electron thermal conductivity owing to collisions between degenerate electrons P. S. Shternin and D. G. Yakovlev Ioﬀe Physical Technical Institute, Politekhnich

In [5]:
from unstructured.partition.pdf import partition_pdf
import base64
from IPython.display import Image, display
import unstructured
import pdfminer

/Users/navdeep/Documents/Hackathon/RollAmp/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
output_path = "./downloads/"

def display_base64_image(base64_code):
    """Display a base64-encoded image."""
    image_data = base64.b64decode(base64_code)
    display(Image(data=image_data))

def process_pdf(file_path):
    """Process a single PDF and extract text, images, and tables."""
    chunks = partition_pdf(
        filename=file_path,
        infer_table_structure=True,            # Extract tables
        strategy="hi_res",                     # Mandatory to infer tables
        extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
        image_output_dir_path=output_path,     # Directory to save images
        extract_image_block_to_payload=True,   # Extract base64 for API usage
        chunking_strategy="by_title",         # or 'basic'
        max_characters=10000,                  # Defaults to 500
        combine_text_under_n_chars=2000,       # Defaults to 0
        new_after_n_chars=6000,
    )

    tables = []
    texts = []
    images = []

    # Separate tables, text, and extract images
    for chunk in chunks:
        if "Table" in str(type(chunk)):
            tables.append(chunk)
        elif "CompositeElement" in str(type(chunk)):
            texts.append(chunk)
            # Extract images from CompositeElements
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images.append(el.metadata.image_base64)

    return texts, tables, images


In [ ]:
def process_images(images):
    """Simulate image processing and return processed images."""
    # Replace this function with the actual image processing logic
    return [f"processed_{i}" for i, _ in enumerate(images)]

def insert_into_snowflake(data, schema="PUBLIC", table="RESEARCH_PAPERS"):
    """Insert data into Snowflake database."""
    conn = snowflake.connector.connect(
        user='YOUR_USER',
        password='YOUR_PASSWORD',
        account='YOUR_ACCOUNT'
    )
    try:
        cursor = conn.cursor()
        # Create schema if not exists
        cursor.execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")

        # Create table if not exists
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {schema}.{table} (
                id INT AUTOINCREMENT PRIMARY KEY,
                title STRING,
                authors ARRAY,
                summary STRING,
                pdf_url STRING,
                extracted_text STRING,
                extracted_tables ARRAY,
                processed_images ARRAY
            )
        """)

        # Insert data
        for record in data:
            cursor.execute(f"""
                INSERT INTO {schema}.{table} (
                    title, authors, summary, pdf_url, extracted_text, extracted_tables, processed_images
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                record['title'],
                record['authors'],
                record['summary'],
                record['pdf_url'],
                record['extracted_text'],
                record['extracted_tables'],
                record['processed_images']
            ))
        conn.commit()
    finally:
        conn.close()


In [22]:
# data_to_insert = []
# Iterate through all PDFs
# for pdf_path, paper in zip(pdf_paths, papers):
extracted_text=[]
extracted_images=[]
for pdf_path in pdf_paths:
    print(f"Processing PDF: {pdf_path}\n{'-'*40}")

    # Process the PDF
    texts, tables, images = process_pdf(pdf_path)
    textStr=""
    # Display extracted text
    print("Extracted Text:")
    for text in texts:
        textStr+=text.text
        print(text.text[:500], "...\n")  # Print first 500 characters of each text chunk
    extracted_text.append(textStr)
    # Display extracted tables
    print("Extracted Tables:")
    for table in tables:
        print(table.text[:500], "...\n")  # Print first 500 characters of each table

    # Display extracted images
    print("Extracted Images:")
    for img_b64 in images:
        display_base64_image(img_b64)

    print("\n")
    
    # # Process images
    # processed_images = process_images(images)

    # # Prepare data for insertion
    # record = {
    #     "title": paper['title'],
    #     "authors": paper['authors'],
    #     "summary": paper['summary'],
    #     "pdf_url": paper['pdf_url'],
    #     "extracted_text": " ".join([text.text for text in texts]),
    #     "extracted_tables": [table.text for table in tables],
    #     "processed_images": processed_images
    # }
    # data_to_insert.append(record)


Processing PDF: downloads/0102536v1.pdf
----------------------------------------
Extracted Text:
2001

cond-mat/0102536v1 [cond-mat.str-el] 28 Feb

1V arX

Impact of Electron-Electron Cusp on Configuration Interaction Energies

David Prendergast,” M. Nolan,’) Claudia Filippi,®) Stephen Fahy,”) and J.C. Greer®)

®) Department of Physics and») NMRC, University College, Cork, Ireland

(November 26, 2024)

The effect of the electron-electron cusp on the convergence of configuration interaction (CI) wave functions is examined. By analogy with the pseu an effective electron-electron interaction is deve lopotential approach for electron-ion interactions, loped which closely reproduces the scattering of the Coulomb interaction but is smooth and finite at zero electron-electron separation. The exact many-electron wave function for this smooth ef ective interaction has no cusp at zero electron- electron separation. We perform CI and quantum Monte Carlo calculations for He and Be atoms, both with

In [ ]:
# Step 2: Insert data into Snowflake
insert_into_snowflake(data_to_insert)

In [18]:
# Create a Snowflake session
session = sf.connect(
    user=snowflake_user,
    password= snowflake_password,
    account= snowflake_account,
    # "role": snowflake_role,
    # "warehouse": snowflake_warehouse,
    # "database": snowflake_database,
    # "schema": snowflake_schema,
)

# st.write("Connected to Snowflake successfully!")

# def upload_to_stage(papers, stage_name="research_papers_db.public.fomc"):
#     for paper in papers:
#         pdf_path = paper["pdf_path"]
#         pdf_name = os.path.basename(pdf_path)
#         session.file.put(pdf_path, f"@{stage_name}/{pdf_name}", auto_compress=False)

# upload_to_stage(papers)
# print("PDFs uploaded to Snowflake stage successfully.")


In [15]:
%pip install snowflake-ml-python

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 24.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.6 MB/s eta 0:00:00
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.8.0
    Uninstalling anyio-4.8.0:
      Successfully uninstalled anyio-4.8.0
Note: you may need to restart the kernel to use updated packages.


In [16]:
from snowflake.snowpark import Session
from snowflake.ml.utils import connection_params
from snowflake.cortex import ClassifyText, Complete, Summarize
from typing import Dict, Optional, Union, List
from snowflake.snowpark.functions import col, udf, parse_json
from snowflake.snowpark.types import StringType
from dotenv import load_dotenv
from pathlib import Path

import snowflake.connector
import json

In [ ]:
from snowflake.snowpark import Session
from snowflake.cortex import Complete
 
snowflake_config = {
    "account": snowflake_account,
    "user": snowflake_user,
    "password": snowflake_password,
    "role": snowflake_role,
    "warehouse": snowflake_warehouse,
    "database": snowflake_database,
    # "schema": "<your_schema>"
}

session = Session.builder.configs(snowflake_config).create()



/var/folders/kf/zy5yhv3j6_xd7bjfyt4t2vth0000gn/T/ipykernel_50995/3568863117.py:16: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  stream = Complete(


 Snow
fl
ake
 SQL
 is
 a
 dialect
 of
 SQL
 designed
 specifically
 for
 the
 Snow
fl
ake
 Data
 Cloud
,
 a
 cloud
-
based
 data
 ware
h
ousing
 platform
.
 It
 includes
 several
 unique
 features
 and
 extensions
 that
 different
iate
 it
 from
 standard
 SQL
.
 Here
 are
 some
 of
 the
 key
 features
:




1
.
 **
Time
 Travel
**:


  
 -
 All
ows
 you
 to
 query
 data
 as
 it
 existed
 at
 a
 specific
 point
 in
 time
 in
 the
 past
.


  
 -
 Use
ful
 for
 data
 recovery
,
 aud
iting
,
 and
 historical
 analysis
.




2
.
 **
Cl
oning
**:


  
 -
 En
ables
 the
 creation
 of
 zero
-
copy
 cl
ones
 of
 dat
abases
,
 sche
mas
,
 tables
,
 and
 other
 objects
.


  
 -
 Cl
ones
 are
 independent
 copies
 that
 do
 not
 consume
 additional
 storage
 until
 changes
 are
 made
.




3
.
 **
Zero
-
Copy
 Cl
oning
**:


  
 -
 Cl
ones
 do
 not
 consume
 additional
 storage
 until
 changes
 are
 made
,
 making
 it
 efficient
 for
 creating
 copies
 for
 testing
,
 development
,
 or
 backup


In [23]:
# Prompt
prompt_text = f"""
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {extracted_text[0]}

"""


In [24]:
print(prompt_text)


You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: 2001

cond-mat/0102536v1 [cond-mat.str-el] 28 Feb

1V arX

Impact of Electron-Electron Cusp on Configuration Interaction Energies

David Prendergast,” M. Nolan,’) Claudia Filippi,®) Stephen Fahy,”) and J.C. Greer®)

®) Department of Physics and») NMRC, University College, Cork, Ireland

(November 26, 2024)

The effect of the electron-electron cusp on the convergence of configuration interaction (CI) wave functions is examined. By analogy with the pseu an effective electron-electron interaction is deve lopotential approach for electron-ion interactions, loped which closely reproduces the scattering of the Coulomb interaction but is smooth and finite at zero electron-electron separation. The exac

In [ ]:
stream = Complete(
  "mistral-large2",
  prompt_text,
  session=session,
  stream=True)
 
for update in stream:
  print(update,end=" ")

/var/folders/kf/zy5yhv3j6_xd7bjfyt4t2vth0000gn/T/ipykernel_50995/3382494087.py:1: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  stream = Complete(


 The  table  or  text  chunk  discuss es  a  scientific  study  on  the  impact  of  the  electron - elect ron  c us p  on  the  convergence  of  configuration  interaction  ( CI )  wave  functions .  The  study  exam ines  the  effect  by  replacing  the  true  C oul omb  electron - elect ron  interaction  with  a  smooth  and  finite  effective  interaction  that  closely  reprodu ces  the  scattering  of  the  C oul omb  interaction  but  is  smooth  at  zero  electron - elect ron  separation .  The  authors  perform  C I  and  quantum  Monte  Carlo  calculations  for  He  and  Be  atoms  with  both  the  C oul omb  and  the  effective  interaction .  They  find  that  the  convergence  of  the  C I  expansion  for  the  smooth  effective  interaction  is  not  significantly  improved  compared  to  the  C oul omb  interaction ,  indicating  that  the  description  of  the  electron - elect ron  c us p  is  not  a  limiting  factor  for  C I  calculations  within  chemical  accuracy

In [ ]:
def sendToMistral(prompt,session):
    stream = Complete(
    "mistral-large2",
    prompt_text,
    session=session,
    stream=True)    
    